# Setup

## Installing and Importing Libraries

Updating all existing Installing libraries and installing required libraries that doesn't come with google colab.  

**Please run this cell before importing the libraries in order to avoid errors.**

In [1]:
!apt update
!apt install -yq chromium-chromedriver
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [998 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,018 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1

Importing libraries.

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.relative_locator import locate_with
from bs4 import BeautifulSoup

## Connecting to driver and loading website

Establishing connection to browser:

In [33]:
# Set up options for Chrome in headless mode
options = webdriver.ChromeOptions()
options.add_argument('--headless') # allows Chrome to run without a graphical user interface
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Start the WebDriver
driver = webdriver.Chrome(options=options)

### Retrieving the link to the player's profile


The following chunk of code is an example on how to get one link to the player's profile page.

In [ ]:
main_url = "https://www.atptour.com/en/players"
search_term = "roger federer"

try:
    # Step 1: Navigate to the URL
    driver.get(main_url)

    # Step 2: Find the search bar by ID and input the player's name
    driver.implicitly_wait(5)

    search_bar = driver.find_element(By.ID, "playerInput")
    search_bar.send_keys(search_term)

    # Step 3: Wait for the dropdown to appear and find the first 'no-padding' class
    driver.implicitly_wait(5)

    dropdown = driver.find_element(By.ID, "playerDropdown")
    first_option = dropdown.find_element(By.CLASS_NAME,"no-padding")

    # Step 4: Retrieve the link inside the first 'no-padding' class; which is the link to the player's profile
    player_link_class = first_option.find_element(By.TAG_NAME,"a")
    player_link = str(player_link_class.get_attribute("href"))
    print(player_link)

finally:
    # Optional: Close the browser window when you are done
    driver.quit()

The folloing code is a function that takes a list of players names as an argument and returns a list of links to thier profiles.

In [24]:
def retrieving_players_profile_links(players_names: list[str], main_url = "https://www.atptour.com/en/players"):
  """This function takes a list with the names of tennis players and returns a list with links to their profile
  on the ATP website"""
  try:
    players_links_list = []
    for i in range(len(players_names)):
      # Step 1: Navigate to the URL
      driver.get(main_url)

      # Step 2: Find the search bar by ID and input the player's name
      driver.implicitly_wait(5)

      search_bar = driver.find_element(By.ID, "playerInput")
      search_bar.send_keys(players_names[i])

      # Step 3: Wait for the dropdown to appear and find the first 'no-padding' class
      driver.implicitly_wait(5)

      dropdown = driver.find_element(By.ID, "playerDropdown")
      first_option = dropdown.find_element(By.CLASS_NAME,"no-padding")

      # Step 4: Retrieve the link inside the first 'no-padding' class; which is the link to the player's profile
      player_link_class = first_option.find_element(By.TAG_NAME,"a")
      player_link = str(player_link_class.get_attribute("href"))
      # print(player_link)
      players_links_list.append(player_link)

  finally:
    # Optional: Close the browser window when you are done
    driver.quit()
    return players_links_list

An example on how to use the function:

In [28]:
players_profile_links = retrieving_players_profile_links(players_names=["roger federer","rafael nadal"])
players_profile_links

['https://www.atptour.com/en/players/roger-federer/f324/overview',
 'https://www.atptour.com/en/players/rafael-nadal/n409/overview']

---

### Retrieving player's data from his profile

In [44]:
player_variabels = []
try:
  # Step 1: Navigate to the URL
  driver.get(player_link)

  # Step 2: Getting the html element that contains the player's static data (birthdate, year turned pro, weight, etc.)
  driver.implicitly_wait(5)

  players_data = driver.find_element(By.CLASS_NAME, "player-profile-hero-table")
  players_data_html = BeautifulSoup(players_data.get_attribute("innerHTML"), "html.parser")

  player_attri = players_data_html.find_all(class_=["table-big-value","table-value"])
  for el in player_attri:
    player_variabels.append(el.get_text().strip())

  player_variabels[0] = player_variabels[0][-11:-1]
  print(player_variabels)
finally:
    # Optional: Close the browser window when you are done
    driver.quit()

['1981.08.08', '1998', '187lbs(85kg)', '6\'1"(185cm)', 'Basel, Switzerland', 'Right-Handed, One-Handed Backhand', 'Ivan Ljubicic, Severin Luthi']


In [36]:
player_variabels[0][-11:-1]

'1986.06.03'

In [45]:
def retrieving_players_data_from_profile_links(profile_links: list[str]):

  players_features_df = pd.DataFrame(columns=["Birthdate", "Year_Turned_Pro", "Birth_Place", "Weight", "Height", "Hands", "Coach"])

  for i in range(len(profile_links)):
    player_variabels = []
    # Step 1: Navigate to the URL
    driver.get(profile_links[i])

    # Step 2: Getting the html element that contains the player's static data (birthdate, year turned pro, weight, etc.)
    driver.implicitly_wait(5)

    players_data = driver.find_element(By.CLASS_NAME, "player-profile-hero-table")
    players_data_html = BeautifulSoup(players_data.get_attribute("innerHTML"), "html.parser")

    player_attri = players_data_html.find_all(class_=["table-big-value","table-value"])

    for el in player_attri:
      player_variabels.append(el.get_text().strip())

    player_variabels[0] = player_variabels[0][-11:-1]

    players_features_df.loc[len(players_features_df.index)] = player_variabels

  return players_features_df

In [46]:
retrieving_players_data_from_profile_links(players_profile_links)

,Birthdate,Year_Turned_Pro,Birth_Place,Weight,Height,Hands,Coach
0,1981.08.08,1998,187lbs(85kg),"6'1""(185cm)","Basel, Switzerland","Right-Handed, One-Handed Backhand","Ivan Ljubicic, Severin Luthi"
1,1986.06.03,2001,187lbs(85kg),"6'1""(185cm)","Manacor, Mallorca, Spain","Left-Handed, Two-Handed Backhand","Carlos Moya, Marc Lopez, Gustavo Marcaccio"
